In [1]:
!pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.57.4
    Uninstalling openai-1.57.4:
      Successfully uninstalled openai-1.57.4


In [2]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [3]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 58.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchain-0.3.12:
      Successfully uninstalled langchain-0.3.12


In [4]:
import openai
import gradio as gr
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

# **Important Security Notice:**
# Hardcoding API keys is not secure. This is for demonstration purposes only.
# Replace "your_openai_api_key" with your actual API key.
openai.api_key = "Add_your_openai_api_key_here"

# Define the system prompt
system_prompt = (
    "You are a compassionate and knowledgeable AI doctor. Your task is to assist users by interpreting their "
    "medical symptoms, reports, and queries with clarity and precision. Always provide accurate, evidence-based "
    "advice while emphasizing the importance of consulting a licensed healthcare professional for diagnosis or "
    "treatment. Your responses should be empathetic, easy to understand, and structured to address medical concerns effectively."
)

# Initialize the language model with the appropriate parameters
llm = ChatOpenAI(temperature=0, openai_api_key=openai.api_key, model="gpt-3.5-turbo")
memory = ConversationBufferMemory()

# Define the prompt template using 'input' instead of 'question'
prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_prompt),
    HumanMessagePromptTemplate.from_template("{input}\n\nHistory: {history}")  # Use 'input' here
])

# Create the ConversationChain with the updated prompt
conversation = ConversationChain(llm=llm, memory=memory, prompt=prompt)

# Function to interact with LangChain and handle memory
def ask_python_question_with_memory(question):
    if question.lower() == "stop":
        return "Exiting the assistant. Have a great day!"

    try:
        # Pass the question directly as a string
        response = conversation.run(question)
        return response
    except Exception as e:
        return f"An error occurred: {e}"


<ipython-input-4-00fd10585e28>:22: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0, openai_api_key=openai.api_key, model="gpt-3.5-turbo")
<ipython-input-4-00fd10585e28>:23: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()
<ipython-input-4-00fd10585e28>:32: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead

In [5]:
# Create the Gradio interface
with gr.Blocks() as ui:
    gr.Markdown("## Medical Assistant ")
    with gr.Row():
        question_input = gr.Textbox(label="Enter your question", placeholder="Type your question here...")
    with gr.Row():
        answer_output = gr.Textbox(label="Answer", interactive=False)
    with gr.Row():
        submit_button = gr.Button("Get Answer")

    # Set up event listeners
    submit_button.click(ask_python_question_with_memory, inputs=[question_input], outputs=[answer_output])

# Launch the app
ui.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://88a243a85d0a1fd479.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
